In [2]:
# imports de bibliotecas
import numpy as np
import mne
from scipy.signal import welch
import matplotlib.pyplot as plt

In [3]:
def print_graphs(X):
    for i in range(X.shape[1]):
        plt.plot(X[:,i])
    plt.title('Domínio do tempo')
    plt.show()

    for i in range(X.shape[1]):
        plt.psd(X[:,i], Fs=250)
    plt.title('Domínio da frequência')
    plt.show()

    for i in range(X.shape[1]):
        plt.specgram(X[i,:], Fs=250)
    plt.title('Espectrograma')
    plt.show()

In [4]:
# definição do info (MNE)

n_channels = 8
ch_types = ['eeg'] * n_channels
sfreq = 250
ch_names = ["F3", "Fz", "F4", "C3", "Cz", "C4", "P3", "P4"]
info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)
info.set_montage("standard_1020")

Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,11 points
Good channels,8 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,250.00 Hz
Highpass,0.00 Hz
Lowpass,125.00 Hz


In [5]:
# carregamento e organização dos dados de: Maria

ga = [
  '../dataset/maria/ia/OpenBCI-RAW-2023-09-28_16-51-25.txt', # aula: 00:08 - 20:02, teste: 20:23 - 23:10
]
tf = [
  '../dataset/maria/tf/OpenBCI-RAW-2023-11-07_13-17-01.txt', # basal: 3:28, teste: 4:28 - 12:29
]
sr = 250                  # taxa de amostragem

ga_ob = [np.loadtxt(txt, delimiter=',', skiprows=5, usecols=range(1, 9)) for txt in ga]
data_ga = ga_ob[0][8*sr:1202*sr,:]
data_ga_test = ga_ob[0][1223*sr:1390*sr,:]

tf_ob = [np.loadtxt(txt, delimiter=',', skiprows=5, usecols=range(1, 9)) for txt in tf]
data_basal = tf_ob[0][208*sr:268*sr,:]
data_tf = tf_ob[0][268*sr:749*sr,:]

# Insulina Ativa
x_ga = mne.io.RawArray(data_ga.T, info)
# Insulina Ativa Teste
x_ga_test = mne.io.RawArray(data_ga_test.T, info)
# Teste final
x_tf = mne.io.RawArray(data_tf.T, info)
# Basal
x_basal = mne.io.RawArray(data_basal.T, info)

# x_ga.set_eeg_reference(ref_channels='average') # CAR
# x_ga_test.set_eeg_reference(ref_channels='average') # CAR
# x_tf.set_eeg_reference(ref_channels='average') # CAR
# x_basal.set_eeg_reference(ref_channels='average') # CAR

X = {
  'ga': x_ga,
  'ga_test': x_ga_test,
}

Creating RawArray with float64 data, n_channels=8, n_times=298500
    Range : 0 ... 298499 =      0.000 ...  1193.996 secs
Ready.
Creating RawArray with float64 data, n_channels=8, n_times=41750
    Range : 0 ... 41749 =      0.000 ...   166.996 secs
Ready.
Creating RawArray with float64 data, n_channels=8, n_times=120250
    Range : 0 ... 120249 =      0.000 ...   480.996 secs
Ready.
Creating RawArray with float64 data, n_channels=8, n_times=15000
    Range : 0 ... 14999 =      0.000 ...    59.996 secs
Ready.


In [6]:
# filtragem de todos os dados

for key in X:
    X[key].notch_filter(freqs=60)
    X[key].filter(l_freq=4, h_freq=100)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband edge: 60.65 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 60.90 Hz)
- Filter length: 1651 samples (6.604 s)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband e

In [7]:
time = {
  'ga': X['ga'].times[-1],
  'ga_test': X['ga_test'].times[-1],
}

sr = 250                  # taxa de amostragem
jump = 5                  # 5s de buffer
size = sr * jump          # quantidade de pontos avalidados
nperseg = 128             # Número de pontos por segmento
noverlap = nperseg // 2   # Quantidade de sobreposição entre segmentos

# Definir os limites das bandas de frequência (em Hz)
theta_band = (4, 8)       # Theta:  4 - 8   Hz
alpha_band = (8, 13)      # Alpha:  8 - 13  Hz
beta_band = (13, 30)      # Beta:  13 - 30  Hz
gamma_band = (30, 100)    # Gamma: 30 - 100 Hz

allBandBuffers = []
bufferBand = []

allBuffers = []
buffer = []
buffer_size = 5
start_time = 0
stop_time = start_time + buffer_size
beta_intervals = {}
data_names = ('ga', 'ga_test')

sample = {
  'focus':[],
  'not_focus':[]
}

for k, data in enumerate(X.values()):
  beta_intervals[k] = []
  for i in range(0, int(time[data_names[k]])-buffer_size-1):
    results = [0,0,0,0]
    start_time = i
    stop_time = start_time + buffer_size

    cut = data.get_data(tmin=start_time, tmax=stop_time)
    freqs, psd = welch(cut, fs=sr, nperseg=nperseg, noverlap=noverlap)

    avg = np.average(psd, axis=0)

    # Encontrar os índices correspondentes às frequências de interesse
    theta_idxs = np.where((freqs >= theta_band[0]) & (freqs <= theta_band[1]))[0]
    alpha_idxs = np.where((freqs >= alpha_band[0]) & (freqs <= alpha_band[1]))[0]
    beta_idxs =  np.where((freqs >=  beta_band[0]) & (freqs <=  beta_band[1]))[0]
    gamma_idxs = np.where((freqs >= gamma_band[0]) & (freqs <= gamma_band[1]))[0]

    # Calcular a potência em cada banda de frequência
    bands = [
      np.sum(avg[theta_idxs]), 
      np.sum(avg[alpha_idxs]), 
      np.sum(avg[ beta_idxs]), 
      np.sum(avg[gamma_idxs])
    ]
    results[np.argmax(bands)] += 1
    if len(buffer) == buffer_size:

      del buffer[0]
    buffer.append(results)
    copyBuffer = buffer.copy()
    
    acc = np.array([0,0,0,0])
    for col in copyBuffer:
      acc = acc + np.array(col)

    #fluxo novo
    if(acc[2] > 2 or acc[3] > 2):
      sample['focus'].append(avg)
    else:
      sample['not_focus'].append(avg)
    #fluxo antigo
    if(acc[2] > 2):
      beta_intervals[k].append([start_time,stop_time, acc, cut])
    allBuffers.append(copyBuffer)

[]
[[1, 0, 0, 0]]
[[1, 0, 0, 0], [1, 0, 0, 0]]
[[1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0]]
[[1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0]]
[[1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1]]
[[1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 1]]
[[1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1]]
[[1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1]]
[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1]]
[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1]]
[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1]]
[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1]]
[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1]]
[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1]]
[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1]]
[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1]]
[

In [8]:
for k in beta_intervals:
  print('name:', data_names[k])
  for times in beta_intervals[k]:

    # data = X[data_names[k]].get_data(tmin=times[0], tmax=times[1])
    data =  times[3] #X[data_names[k]].get_data(tmin=times[0], tmax=times[1])
    data = data.transpose() 
    print(f'time interval: ({times[0]}s - {times[1]}s)')
    # print_graphs(data)


name: ga
time interval: (349s - 354s)
time interval: (350s - 355s)
time interval: (351s - 356s)
time interval: (352s - 357s)
time interval: (810s - 815s)
time interval: (811s - 816s)
time interval: (812s - 817s)
time interval: (813s - 818s)
time interval: (1134s - 1139s)
time interval: (1135s - 1140s)
time interval: (1136s - 1141s)
time interval: (1137s - 1142s)
time interval: (1138s - 1143s)
time interval: (1139s - 1144s)
name: ga_test


In [9]:
reducedBuffers = []
for buffer in allBuffers:
  acc = np.array([0,0,0,0])
  for item in buffer:
    acc = acc + np.array(item)
  reducedBuffers.append(acc)


print('len-reducedBuffers: ',len(reducedBuffers))
for buffer in reducedBuffers:
  print(buffer)

len-reducedBuffers:  1347
[1 0 0 0]
[2 0 0 0]
[3 0 0 0]
[4 0 0 0]
[4 0 0 1]
[3 0 0 2]
[2 0 0 3]
[1 0 0 4]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[0 0 0 5]
[1 0 0 4]
[2 0 0 3]
[3 0 0 2]
[4 0 0 1]
[5 0 0 0]
[5 0 0 0]
[5 0 0 0]
[5 0 0 0]
[5 0 0 0]
[5 0 0 0]
[5 0 0 0]
[5 0 0 0]
[5 0 0 0]
[5 0 0 0]
[5 0 0 0]
[5 0 0 0]
[5 0 0 0]
[5 0 0 0]
[5 0 0 0]
[5 0 0 0]
[5 0 0 0]
[5 0 0 0]
[5 0 0 0]
[5 0 0 0]
[4 0 0 1]
[3 0 0 2]
[3 0 0 2]
[3 0